In [ ]:
#%pip install 

In [1]:
# Import the required libraries
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image

class AutoToggle:
    text = "INACTIVE"
    color = "red"
    fuel = 33
    scan_fire = "none"

nToggle = AutoToggle()

In [59]:
text_color = '#e08533'
bg_color = 'black'
bg_hover = '#e08533'
print(nToggle.scan_fire)

# Create an instance of tkinter frame or window
win=tk.Tk()
#set the back ground color
win.config(background=bg_color)

var = tk.IntVar()
active_text=tk.StringVar()
active_text.set("INACTIVE")

fuel_var=tk.IntVar()
#fuel_var.set(nToggle.fuel)

# Set the size of the window
win.title('Elite Auto Pilot')
win.geometry("280x240")

#set auto pilot on
def auto_on():
   if active_text.get() == "ACTIVE":
      active_text.set("INACTIVE")
      auto_status["fg"] = 'red'
      print('inactive')
   else:
      active_text.set("ACTIVE")
      auto_status["fg"] = 'green'
      print('active')

#set the fuel scoop percentange
def set_scoop(none):
   fuel_var.set(scoop_slide.get())
   nToggle.fuel = scoop_slide.get()

#set the discover scan button primary or secondary
def set_scan():
   AutoToggle.scan_fire = var.get()
   if AutoToggle.scan_fire == 1:
      fire_btn = "primary"
   else:
      fire_btn = "secondary"
   nToggle.scan_fire=fire_btn
   selection = nToggle.scan_fire + " selected"
   print(selection)

#event handler on button hover
def on_enter_auto(event):
   auto_status["background"] = bg_hover

def on_leave_auto(event):
   auto_status["background"] = bg_color

#functions to keep the scale object active to give it a different color
def active_delay():
   scoop_slide["state"] = 'active'

def keep_active(event):
   win.after(2, active_delay)

#create the autopilot statyic text
AutoText=tk.Label(
   win, 
   text="Auto Pilot", 
   fg=text_color, 
   font=("Helvetica", 16), 
   background=bg_color
   )

AutoText.grid(
   row = 0, 
   column = 0, 
   padx = (20,0), 
   pady = (20, 5), 
   sticky = 'w'
   )

#create auto pilot toggle button default inactive
auto_status=tk.Button(
   win, 
   text="INACTIVE", 
   textvariable=active_text, 
   fg = 'red', 
   font=('Aerial 18'), 
   height=1, 
   width=8, 
   activebackground=bg_hover, 
   background=bg_color, 
   borderwidth=0,
   command=auto_on
   )

auto_status.grid(
   row = 0, 
   column = 1, 
   pady = (20,5)
   )

#check for button hover
auto_status.bind("<Enter>", on_enter_auto)
auto_status.bind("<Leave>", on_leave_auto)

fuel_text = tk.Label(
   win,
   text="Fuel Scoop Threshold", 
   fg=text_color, font=("Helvetica", 10), 
   background=bg_color
)

fuel_text.grid(
   row=2, 
   column=0, 
   padx=(20,5), 
   #sticky="sw",
   columnspan=2
   )

#create the fuel scoop percentage
scoop_slide = tk.Scale(win, 
   from_=0, to=100, 
   orient="horizontal",
   length=200,
   fg=text_color, 
   background=bg_color, 
   borderwidth=0, 
   activebackground='white', 
   troughcolor=text_color, 
   highlightthickness=5, 
   highlightcolor=bg_color, 
   highlightbackground=bg_color,
   state='active',
   foreground=text_color,
   showvalue=0,
   variable=fuel_var,
   command=set_scoop
   )

scoop_slide.bind("<Leave>", keep_active)
scoop_slide.bind("<Enter>", keep_active)
   

scoop_slide.set(nToggle.fuel)
scoop_slide.grid(
   row=3, 
   column=0,
   columnspan=2,
   padx=(20,5),  
   sticky="w"
   )

#display the current fuel scoop percentage
scoop_percent = tk.Label(
   win,
   text=str(nToggle.fuel)+"%",
   textvariable=fuel_var,
   fg=text_color, font=("Helvetica", 10),
   background=bg_color
)

scoop_percent.grid(
   row=4, 
   column=0, 
   padx=(20,0),
   sticky="e" 
   )

#add the percent sign
scoop_tick = tk.Label(
   win,
   text="%",
   fg=text_color, font=("Helvetica", 10),
   background=bg_color
)

scoop_tick.grid(
   row=4, 
   column=1, 
   sticky="w" 
   )

#create the set fuel percentage button


#create the scan label text
scan_label = tk.Label(
   win, 
   text="Discovery Scanner Layout", 
   fg=text_color, font=("Helvetica", 12), 
   background=bg_color
   )

scan_label.grid(
   row=5, 
   column=0, 
   padx=(20,5), 
   pady=(5,5), 
   #sticky="w", 
   columnspan=2
   )

#create radio button to select primary or secondary fire
R1 = tk.Radiobutton(
   win, 
   text="Primary", 
   variable=var, 
   fg=text_color, 
   value=1, 
   bg=bg_color, 
   command=set_scan,
   indicatoron=0,
   )

R1.grid(
   row=6, 
   column=0, 
   padx=(5,5),
   pady=(5,5), 
   sticky="e"
   )

R2 = tk.Radiobutton(
   win, 
   text="Secondary", 
   variable=var, 
   fg=text_color, 
   value=2, 
   bg=bg_color, 
   command=set_scan,
   indicatoron=0
   )

R2.grid(
   row=6, 
   column=1,
   padx=(5,5),
   pady=(5,5), 
   sticky="w" 
   )

if nToggle.scan_fire == "primary":
   R1.select()
if nToggle.scan_fire == "secondary":
   R2.select()

win.mainloop()

primary


AttributeError: module 'tkinter' has no attribute 'End'

In [48]:
import tkinter as tk
from tkinter import ttk

root = tk.Tk()

mytext = tk.StringVar(value='test ' * 30)

myframe = ttk.Frame(root)
myentry = ttk.Entry(myframe, textvariable=mytext, state='readonly')
myscroll = ttk.Scrollbar(myframe, orient='horizontal', command=myentry.xview)
myentry.config(xscrollcommand=myscroll.set)

myframe.grid()
myentry.grid(row=1, sticky='ew')
myscroll.grid(row=2, sticky='ew')

root.mainloop()